In [1]:
import numpy as np
import pandas as pd

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.tree import DecisionTreeClassifier

In [3]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin', 'Fare'], inplace=True)

In [5]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Embarked  889 non-null    object 
dtypes: float64(1), int64(4), object(2)
memory usage: 48.9+ KB


In [7]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [15]:
x = df.drop(columns=['Survived'])
y = df['Survived']
x.head()

,Pclass,Sex,Age,SibSp,Parch,Embarked
0,3,male,22.0,1,0,S
1,1,female,38.0,1,0,C
2,3,female,26.0,0,0,S
3,1,female,35.0,1,0,S
4,3,male,35.0,0,0,S


In [9]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [10]:
# filling missing vals

tnf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embark', SimpleImputer(strategy='most_frequent'), [5])
], remainder='passthrough')

In [11]:
# encoding cols

tnf2 = ColumnTransformer([
    ('ohe_sex_embark', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [1,5])
], remainder='passthrough')

In [17]:
sample = tnf2.fit_transform(x_train)
sample

array([[ 0.,  1.,  0., ..., 25.,  0.,  0.],
       [ 1.,  0.,  0., ..., 36.,  0.,  0.],
       [ 1.,  0.,  0., ...,  5.,  1.,  2.],
       ...,
       [ 1.,  0.,  0., ..., 58.,  0.,  0.],
       [ 0.,  1.,  0., ..., 32.,  0.,  0.],
       [ 1.,  0.,  0., ..., 44.,  1.,  0.]], shape=(712, 10))

In [18]:
# scaling vals

tnf3 = ColumnTransformer([
    ('scaling', MinMaxScaler(), slice(0,10))
])

In [20]:
# Feature selection

tnf4 = SelectKBest(score_func=chi2, k=8)

In [21]:
# Decision tree                 ------- doesnot get affected by multicolinearity

tnf5 = DecisionTreeClassifier()

# Pipeline

In [22]:
pipe = Pipeline(steps=[
    ('tnf1', tnf1),
    ('tnf2', tnf2),
    ('tnf3', tnf3),
    ('tnf4', tnf4),
    ('tnf5', tnf5)
])

In [23]:
# Alternate
pipes = make_pipeline(tnf1, tnf2, tnf3, tnf4, tnf5)

In [24]:
pipe

Pipeline(steps=[('tnf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embark',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [5])])),
                ('tnf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embark',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 5])])),
                ('tnf3',
                 ColumnTransformer(transformers=[('scaling', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('tnf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001888A9109A0>)),
                ('tnf5', DecisionTreeClassifier())])

In [25]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('tnf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embark',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [5])])),
                ('tnf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embark',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 5])])),
                ('tnf3',
                 ColumnTransformer(transformers=[('scaling', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('tnf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001888A9109A0>)),
                ('tnf5', DecisionTreeClassifier())])

In [29]:
# prediction 
pred = pipe.predict(x_test)

In [30]:
pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0])

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.5865921787709497

In [48]:
pipe.named_steps['tnf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embark', SimpleImputer(strategy='most_frequent'), [5]),
 ('remainder',
  FunctionTransformer(accept_sparse=True, check_inverse=False,
                      feature_names_out='one-to-one'),
  [0, 1, 3, 4])]

In [58]:
new_pipe = Pipeline([
    ('tnf1', tnf1),
    ('tnf2', tnf2),
    ('tnf3', tnf3),
    ('tnf5', tnf5)
])

In [59]:
new_pipe.fit(x_train, y_train)

Pipeline(steps=[('tnf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embark',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [5])])),
                ('tnf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embark',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 5])])),
                ('tnf3',
                 ColumnTransformer(transformers=[('scaling', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('tnf5', DecisionTreeClassifier())])

In [60]:
prediction = new_pipe.predict(x_test)

In [61]:
accuracy_score(y_test, prediction)

0.5865921787709497